In [1]:
'''
This notebook implements a batch run for OFAT analysis
Packages
'''
from model import *
import csv
from mesa.batchrunner import BatchRunner
import pandas as pd
import numpy as np
import tqdm as tqdm
import seaborn as sns
import matplotlib.pyplot as plt
import time
from SALib.sample import saltelli
from SALib.analyze import sobol
from itertools import combinations
from joblib import Parallel, delayed, cpu_count

In [2]:
def run_model(steps, kwargs):
    model = HousingMarket(**kwargs)
    model.run_model(steps)
    data = model.datacollector.get_model_vars_dataframe()
    return data['Mean House Price'][-1]

In [3]:
kwargs = {}

In [4]:
results = Parallel(n_jobs= 10)(delayed(run_model)(steps=10000, kwargs = {})
                       for i in range(10))

In [13]:
import csv

with open('outputs/percentage_free_houses150.csv', 'w') as f: 
    write = csv.writer(f) 
    write.writerows([results])

In [ ]:
import matplotlib.pyplot as plt
plt.hist(results)
plt.title("Standard model")
plt.savefig("outputs/banking_hist_group0.png", dpi=300)

In [ ]:
results1 = []
with open('outputs/banking_experiment_group1.csv', newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
    for row in spamreader:
        results1.append(row)

In [ ]:
plt.hist(list(np.array(results1[0]).astype(float)))
plt.title("More mortgage for long-time renters")
plt.savefig("outputs/banking_hist_group1.png", dpi=300)

## Global Sensitivy Analysis

In [ ]:
# YOUR NAME HERE
name = "Julius"
assert name in ['Martina', "Nils", "Sebastiaan", "Alex", "Julius", "extra"], "please fill in a name of a group member"

In [ ]:
def run_SOBOL(values, problem, max_steps, counter):
    # build kwargs
    kwargs = {}
    for i, name in enumerate(problem['names']):
        kwargs[name] = values[i]
    model = HousingMarket(**kwargs)
    model.run_model(max_steps)
    data = model.datacollector.get_model_vars_dataframe()
        
    # add run data to the dictionary
    kwargs['Run'] = counter
    kwargs['Gini'] = data['Gini'].iloc[-1]

    return pd.DataFrame(pd.Series(kwargs)).T

In [ ]:
replicates = 30
max_steps = 1000
distinct_samples = 64

In [ ]:
# read saltelli samples
param_values = pd.read_csv(f"saltelli_samples/param_values_{name}", index_col=0)

In [ ]:
import math
problem = {
    'num_vars': 8,
    'names': ['savings_lower', 'savings_upper', 'price_lower', 'price_upper', 
              'payoff_perc_freehold', 'inflation', 'chi_parameter', 'bank_income_multiplier'],
    'bounds': [[1, 40_000], [50_000, 100_000], [10_000, 90_000], [100_000, 1_000_000],
              [0.001,0.0025],[0.01,0.03],[5,7.5],[6,9]]    
}
assert len(param_values) == math.floor(distinct_samples*(2*problem['num_vars']+2)/5), "The amount of samples is incorrect"

In [ ]:
from tqdm import notebook
results = Parallel(n_jobs=cpu_count())(delayed(run_SOBOL)(param_values.iloc[i%len(param_values),:], problem, max_steps, 
                                                i)
                       for i in notebook.tqdm(range(len(param_values)*replicates)))

df_global = pd.concat(results)
df_global.to_csv(f"sobol_results/results_{name}")